In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [17]:
# Results

modes = ['AE','classall','classred','syllall','syllred','phonall','phonred']

for mode in modes:

    accuracies = np.load('results/Accuracies_' + mode + '_MLP_1024.npy')

    frame_sizes = ['1024']
    kernel_heights = [3, 5, 7]
    kernel_widths = [3, 5, 7]

    num_it = 10

    accuracies_allpart = np.mean(np.mean(accuracies,axis=-1),axis=-1)

    print('')
    print(mode)
    print('')
    
    for a in range(len(frame_sizes)):
        for b in range(len(kernel_heights)):
            for c in range(len(kernel_widths)):
                print([frame_sizes[a],kernel_heights[b],kernel_widths[c]])
                print(accuracies_allpart[a,b,c])
        
    print('')
    print(np.sum(accuracies_allpart)/accuracies_allpart.size)
    print('')


AE

['1024', 3, 3]
0.765068537316152
['1024', 3, 5]
0.6677088327373245
['1024', 3, 7]
0.7224178996469293
['1024', 5, 3]
0.7480703336851938
['1024', 5, 5]
0.6786070263412382
['1024', 5, 7]
0.629534057953528
['1024', 7, 3]
0.7586087412067821
['1024', 7, 5]
0.7170655051512379
['1024', 7, 7]
0.6366907470992633

0.7026412979041833


classall

['1024', 3, 3]
0.7569293568176885
['1024', 3, 5]
0.6694508189734603
['1024', 3, 7]
0.7200251017298017
['1024', 5, 3]
0.7421604956899369
['1024', 5, 5]
0.675013456294047
['1024', 5, 7]
0.6393775391791546
['1024', 7, 3]
0.761903776973486
['1024', 7, 5]
0.7147682961076498
['1024', 7, 7]
0.631429497152567

0.701228704324199


classred

['1024', 3, 3]
0.7625383581433977
['1024', 3, 5]
0.676036487932184
['1024', 3, 7]
0.7124243959784508
['1024', 5, 3]
0.7384048848279886
['1024', 5, 5]
0.6734643779295897
['1024', 5, 7]
0.6286789413541556
['1024', 7, 3]
0.7536978683301382
['1024', 7, 5]
0.711714138037392
['1024', 7, 7]
0.6435079194073167

0.7000519302156237



In [18]:
# Results

modes = ['AE','classall','classred','syllall','syllred','phonall','phonred']

for mode in modes:

    accuracies = np.load('results/Accuracies_Also_Valid_' + mode + '_MLP_1024.npy')

    frame_sizes = ['1024']
    kernel_heights = [3, 5, 7]
    kernel_widths = [3, 5, 7]

    num_it = 10

    accuracies_allpart = np.mean(np.mean(accuracies,axis=-1),axis=-1)

    print('')
    print(mode)
    print('')
    
    for a in range(len(frame_sizes)):
        for b in range(len(kernel_heights)):
            for c in range(len(kernel_widths)):
                print([frame_sizes[a],kernel_heights[b],kernel_widths[c]])
                print(accuracies_allpart[a,b,c])
        
    print('')
    print(np.sum(accuracies_allpart)/accuracies_allpart.size)
    print('')


AE

['1024', 3, 3]
0.770115571438185
['1024', 3, 5]
0.6732477272493224
['1024', 3, 7]
0.729056068870255
['1024', 5, 3]
0.7552513980214686
['1024', 5, 5]
0.6849810823313488
['1024', 5, 7]
0.6367822752825757
['1024', 7, 3]
0.7662868140819008
['1024', 7, 5]
0.7242708431750804
['1024', 7, 7]
0.6465352210991009

0.7096141112832486


classall

['1024', 3, 3]
0.7622680308722499
['1024', 3, 5]
0.6759519463577071
['1024', 3, 7]
0.726506032911736
['1024', 5, 3]
0.7486347569249457
['1024', 5, 5]
0.6830444534308322
['1024', 5, 7]
0.6462676218882969
['1024', 7, 3]
0.7686223677172573
['1024', 7, 5]
0.7212783436961044
['1024', 7, 7]
0.6408186536365699

0.7081546897150778


classred

['1024', 3, 3]
0.7681581630490573
['1024', 3, 5]
0.6821623402109782
['1024', 3, 7]
0.7175671134172552
['1024', 5, 3]
0.7452919443959871
['1024', 5, 5]
0.6803463542064466
['1024', 5, 7]
0.6359062094681758
['1024', 7, 3]
0.7608262204341855
['1024', 7, 5]
0.7176833406178084
['1024', 7, 7]
0.652139613753029

0.70667569995032

In [13]:
accuracies_allpart

array([[[0.77011557, 0.67324773, 0.72905607],
        [0.7552514 , 0.68498108, 0.63678228],
        [0.76628681, 0.72427084, 0.64653522]]])